# Simple Motion Definition and Simulation

In [ ]:
using KomaMRI # hide
sys = Scanner() # hide

This tutorial illustrates how we can add simple motion to phantoms.
We will also see how phantoms can be stored and loaded from $.phantom$ files.

First, we load a static 3D brain phantom:

In [ ]:
obj1 = brain_phantom3D()
obj1.Δw .= 0 # Removes the off-resonance
p1 = plot_phantom_map(obj1, :T2 ; height=400)

Now, we will make a copy of the phantom, to which we will add Rotation Motion
to recreate the patient's movement inside the scanner.

In [ ]:
obj2 = copy(obj1)
obj2.motion = SimpleMotion([Rotation(t_start=0.0, t_end=0.25, pitch=15.0, roll=0.0, yaw=45.0)])
p2 = plot_phantom_map(obj2, :T2 ; height=400, intermediate_time_samples=4)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*